
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# 3.4 Lab - Manipulate and Analyze a Table

### Duration: ~15-20 minutes

In this lab, you will manipulate a **bronze layer** table to create a **silver layer** table. Then, you will create a scheduled **materialized view** to perform some simple analysis on the table and refresh it daily.


### Objectives
- Apply basic data transformation techniques to clean and structure raw data for further analysis
- Combine data from multiple sources and perform necessary operations to derive meaningful insights
- Create and manage views that aggregate data and automate refresh processes for improved and more efficient analysis.

## REQUIRED - SELECT A SHARED SQL WAREHOUSE

Before executing cells in this notebook, please select the **SHARED SQL WAREHOUSE** in the lab. Follow these steps:

1. Navigate to the top-right of this notebook and click the drop-down to select compute (it might say **Connect**). Complete one of the following below:

   a. Under **Recent resources**, check to see if you have a **shared_warehouse SQL**. If you do, select it.

   b. If you do not have a **shared_warehouse** under **Recent resources**, complete the following:

    - In the same drop-down, select **More**.

    - Then select the **SQL Warehouse** button.

    - In the drop-down, make sure **shared_warehouse** is selected.

    - Then, at the bottom of the pop-up, select **Start and attach**.

<br></br>
   <img src="../Includes/images/sql_warehouse.png" alt="SQL Warehouse" width="600">

## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

### IMPORTANT LAB INFORMATION

Recall that your lab setup is created with the [0 - REQUIRED - Course Setup]($../0 - REQUIRED - Course Setup and Data Discovery) notebook. If you end your lab session or if your session times out, your environment will be reset, and you will need to rerun the Course Setup notebook.

In [0]:
%run ../Includes/3.4-Classroom-Setup

table_status
"Lab Check, all tables available for your lab!"


num_affected_rows,num_inserted_rows
50,50


num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
1467,1467,0


num_affected_rows,num_inserted_rows


Run the following cell to set and view your default catalog and schema. Confirm that your default catalog is **dbacademy** and your schema is **labuser** (this uses the `DA.schema_name` variable created in the classroom setup script).

The default catalog and schema are pre-configured for you in the cell below. This eliminates the need to specify the three-level name for your tables (i.e., catalog.schema.table). All tables will be read from and written to your **dbacademy.labuser** schema.

In [0]:
-- Change the default catalog/schema
USE CATALOG dbacademy;
USE SCHEMA IDENTIFIER(DA.schema_name);


-- View current catalog and schema
SELECT 
  current_catalog(), 
  current_schema(), 
  DA.schema_name AS variable_value   -- Display DA.schema_name variable value

current_catalog(),current_schema(),variable_value
dbacademy,labuser10602209_1756883952,labuser10602209_1756883952


## B. Lab Scenario

You are an analyst at DB Inc., and you’ve been assigned the task of working with customer data stored in several JSON files. The data is located in the volume **dbacademy_retail.v01.retail-pipeline.customers.stream_json**. Your goal is to clean, transform, and enrich the raw data and create two new objects:

- **Silver table**: This table will contain curated, cleaned, and slightly transformed data. It serves as an intermediate stage between raw data and the gold-level table.

- **Gold materialized view**: This materialized view will contain aggregated data that should be scheduled to be refreshed every day.

Follow the steps detailed below.

### B1. Ingestion and Exploration

1. Run the following cell to create a table named **customers_lab_bronze** in your **dbacademy.labuser** schema. The data will be sourced from a series of JSON files located in the provided Marketplace shared data volume: `/Volumes/dbacademy_retail/v01/retail-pipeline/customers/stream_json/`


In [0]:
-- Drop the table if it already exists to start from the beginning
DROP TABLE IF EXISTS customers_lab_bronze;

-- Create an empty table
CREATE TABLE customers_lab_bronze;

-- Copy JSON files from a volume into the table, merge the schema
COPY INTO customers_lab_bronze
FROM '/Volumes/dbacademy_retail/v01/retail-pipeline/customers/stream_json/'
FILEFORMAT = JSON
COPY_OPTIONS ('mergeSchema' = 'true');

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
1467,1467,0


2. Run the cell below to query the **customers_lab_bronze** table and preview 10 rows. Notice the following:

   - The **email** column contains the user's email domain.

   - The **state** column contains a U.S. state abbreviation.
   
   - The **timestamp** column contains a Unix timestamp value.

In [0]:
SELECT *
FROM customers_lab_bronze
LIMIT 10;

address,city,customer_id,email,name,operation,state,timestamp,zip_code
568 David Brook Apt. 524,Norwalk,23056,nelsonjoy@example.com,Brent Chavez,NEW,CA,1632417981,45049
741 Wendy Plains Apt. 143,San Francisco,23057,perkinsdeborah@example.net,James Cruz,NEW,CA,1632421305,42872
732 Trujillo Rue,Santa Monica,23058,jmccullough@example.net,Jennifer Christensen,NEW,CA,1632356384,89020
94580 Wolf Path Apt. 323,Albany,23053,alison80@example.com,Robert Craig,UPDATE,GA,1632409038,49094
954 Oconnell Union Apt. 988,New York,23060,ibuck@example.net,Shannon Cochran,NEW,NY,1632371835,35054
07449 Michael Manor,Pawtucket,23061,anthony99@example.com,Michael Norris,NEW,RI,1632409638,93764
73556 Rogers Glens,St. Charles,23062,vcoleman@example.net,Rhonda Thompson,NEW,MO,1632389458,62504
672 Erica Lakes,Perry,23063,sarahgallagher@example.net,Gene Yang DDS,NEW,MI,1632413696,60175
99513 Shari Views Apt. 667,Roswell,23064,kgonzalez@example.com,Valerie Clark,NEW,GA,1632356121,15232
61040 Hernandez Lane,Gaithersburg,23065,dylantaylor@example.net,Tonya Cameron,NEW,MD,1632400610,58640


3. Write a query below to:

   - View the distinct values in the **state** column of the **customers_lab_bronze** table and name the column **distinct_state**.
   
   - Order the result by the new column **distinct_state** in ascending order.

   Run the query and view the results. Confirm that the query returns 48 rows and that the first value is `null`.

In [0]:
SELECT DISTINCT state AS distinct_state
FROM customers_lab_bronze
ORDER BY distinct_state  ASC;
-- Create a temporary view

distinct_state
null
AL
AR
AZ
CA
CO
CT
DC
FL
GA


4. Run the query below to query the **state_lookup** table and preview 10 rows. Notice that the table contains abbreviated state names, which can be used to obtain the full state names.

In [0]:
SELECT *
FROM state_lookup
LIMIT 10;

state_abbreviation,state_name
AL,Alabama
AK,Alaska
AZ,Arizona
AR,Arkansas
CA,California
CO,Colorado
CT,Connecticut
DE,Delaware
FL,Florida
GA,Georgia


### B2. Data Manipulation and Transformation

1. Create the **customer_lab_silver** table by completing the following:

   a. Select the following columns from the **customers_lab_bronze** table:
      - **customer_id**
      - **name**
      - **address**
      - **city**
      - **email**
      - **operation**
      - **zip_code**
      - **state**
   
   b. Create a column named **sign_up_date** by converting the Unix timestamp (**timestamp** column) to a human-readable date format using the following:
      - [from_unixtime function](https://docs.databricks.com/aws/en/sql/language-manual/functions/from_unixtime)

      - [date_format function](https://docs.databricks.com/aws/en/sql/language-manual/functions/date_format)

      **HINT:**
      ```sql
      date_format(from_unixtime(timestamp), 'yyyy-MM-dd HH:mm:ss') AS sign_up_date
      ```

   c. Create a column named **email_domain** that extracts the domain from the email address using the `substring_index` function.
      - [substring_index function](https://docs.databricks.com/aws/en/sql/language-manual/functions/substring_index#:~:text=Returns%20the%20substring%20of%20expr,occurrence%20of%20the%20delimiter%20delim%20.)

      - Example results: **peter@example.org** → **example.org**

      **HINT:**
      ```SQL
      substring_index(cust.email, '@', -1) AS email_domain,
      ```

   d. Perform a LEFT JOIN between the **customers_lab_bronze** and **state_lookup** tables to retain all records from the **customers_lab_bronze** table, creating the **state_name** column from the **state_lookup** table to obtain the full state name where available.
      - **HINT:** Try using the `generate with AI` in another cell to assist you with the join syntax if you need it.

**NOTE:** The solution table has already been created for you in your **labuser** schema if you would like to view it. The table is named **customer_lab_silver_solution**.


In [0]:
CREATE OR REPLACE TABLE customer_lab_silver AS
SELECT
  customer_id,
  name,
  address,
  city,
  email,
  operation,
  zip_code,
  state,
  date_format(from_unixtime(timestamp), 'yyyy-MM-dd HH:mm:ss') AS sign_up_date,
  substring_index(cust.email, '@', -1) AS email_domain,
  state_name
FROM customers_lab_bronze AS cust
LEFT JOIN state_lookup As st ON cust.state = st.state_abbreviation

num_affected_rows,num_inserted_rows


2. Run the cell below to compare your table **customer_lab_silver** to the solution table **customer_lab_silver_solution**. If the tables are identical (column names, all data, etc) then query should return 0 rows. 

If the query returns an error, you will have to fix the **customer_lab_silver** table you created in the previous cell.

In [0]:
SELECT *
FROM customer_lab_silver_solution
EXCEPT
SELECT
  customer_id,
  name,
  address,
  city,
  email,
  operation,
  zip_code,
  state,
  sign_up_date,
  email_domain,
  state_name
FROM customer_lab_silver;

customer_id,name,address,city,email,operation,zip_code,state,sign_up_date,email_domain,state_name


### B3. Create a Materialized View

1. Run the cell below to view 10 rows from your **customer_lab_silver** table.

In [0]:
SELECT *
FROM customer_lab_silver
LIMIT 10;

customer_id,name,address,city,email,operation,zip_code,state,sign_up_date,email_domain,state_name
23056,Brent Chavez,568 David Brook Apt. 524,Norwalk,nelsonjoy@example.com,NEW,45049,CA,2021-09-23 17:26:21,example.com,California
23057,James Cruz,741 Wendy Plains Apt. 143,San Francisco,perkinsdeborah@example.net,NEW,42872,CA,2021-09-23 18:21:45,example.net,California
23058,Jennifer Christensen,732 Trujillo Rue,Santa Monica,jmccullough@example.net,NEW,89020,CA,2021-09-23 00:19:44,example.net,California
23053,Robert Craig,94580 Wolf Path Apt. 323,Albany,alison80@example.com,UPDATE,49094,GA,2021-09-23 14:57:18,example.com,Georgia
23060,Shannon Cochran,954 Oconnell Union Apt. 988,New York,ibuck@example.net,NEW,35054,NY,2021-09-23 04:37:15,example.net,New York
23061,Michael Norris,07449 Michael Manor,Pawtucket,anthony99@example.com,NEW,93764,RI,2021-09-23 15:07:18,example.com,Rhode Island
23062,Rhonda Thompson,73556 Rogers Glens,St. Charles,vcoleman@example.net,NEW,62504,MO,2021-09-23 09:30:58,example.net,Missouri
23063,Gene Yang DDS,672 Erica Lakes,Perry,sarahgallagher@example.net,NEW,60175,MI,2021-09-23 16:14:56,example.net,Michigan
23064,Valerie Clark,99513 Shari Views Apt. 667,Roswell,kgonzalez@example.com,NEW,15232,GA,2021-09-23 00:15:21,example.com,Georgia
23065,Tonya Cameron,61040 Hernandez Lane,Gaithersburg,dylantaylor@example.net,NEW,58640,MD,2021-09-23 12:36:50,example.net,Maryland


2. Create a materialized view named **customers_per_state_gold** that:

   - Counts the number of customers by **state_name** from the **customers_lab_silver** table in a column named **total_customers**.

   - Ensures the materialized view refreshes every day.
   
   - Orders the results by **total_customers** in descending order.

   [CREATE MATERIALIZED VIEW](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-create-materialized-view)

   **HINT:** If you couldn't create the **customers_lab_silver** table correctly, you can use the premade solution table **customer_lab_silver_solution**.

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW customers_per_state_gold 
SCHEDULE EVERY 1 DAY
AS
SELECT
  state_name,
  COUNT(*) AS total_customers
FROM customer_lab_silver
GROUP BY state_name
ORDER BY total_customers DESC;

result
The operation was successfully executed.


3. Run the cell below to view your materialized view. Confirm your first 10 rows of your materialized view match the following values:

    | state_name  | total_customers |
    |-------------|-----------------|
    | California  | 259             |
    | Texas       | 141             |
    | New York    | 98              |
    | Florida     | 69              |
    | Illinois    | 55              |
    | Michigan    | 52              |
    | Ohio        | 50              |
    | null        | 49              |
    | Washington  | 38              |
    | Arizona     | 36              |


In [0]:
SELECT *
FROM customers_per_state_gold;

state_name,total_customers
Maine,1
New Jersey,20
null,49
Arkansas,18
Oregon,26
Arizona,36
South Carolina,19
Idaho,11
Rhode Island,6
Nebraska,10


4. Write the statement to describe detailed information about your materialized view **customers_per_state_gold** to view the refresh information. 

   Confirm that your **# Refresh Information** section looks similar to this solution example:


| **Refresh Information**     |  **Data Type**                      |
|-----------------------------|-----------------------------------|
| **Last Refreshed**           | 2025-03-20T20:45:05Z (will differ)             |
| **Last Refresh Type**        | RECOMPUTED                        |
| **Latest Refresh Status**    | Succeeded                         |
| **Latest Refresh**           | Your specific refresh |
| **Refresh Schedule**         | EVERY 1 DAYS                       |

In [0]:
DESCRIBE EXTENDED customers_per_state_gold;

col_name,data_type,comment
state_name,varchar(100),null
total_customers,bigint,null
,,
# Detailed Table Information,,
Catalog,dbacademy,
Database,labuser10602209_1756883952,
Table,customers_per_state_gold,
Owner,labuser10602209_1756883952@vocareum.com,
Created Time,Thu Sep 04 13:34:05 UTC 2025,
Last Access,UNKNOWN,



&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>